In [1]:
%matplotlib inline
import os
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
def read(data_filepath, usecols):
    dat = pd.read_csv(data_filepath, usecols=[usecols]).drop_duplicates()
    dat.columns = ['text']
    dat.loc[:, 'text'] += '(end)'
    return dat

In [17]:
dat = read('../lyrics_files/allSongs_d.csv', 'lyrics')

In [18]:
dat.head()

,text
0,"I'm tryna put you in the worst mood, ah\nP1 cl..."
1,"Hey, I was doing just fine before I met you\nI..."
2,I used to believe\nWe were burnin' on the edge...
3,Baby I like your style\n\nGrips on your legs\n...
4,"Yeah, yeah\n\nI've been down so long it look l..."


In [19]:
def get_unique_chars(dat):
    unique_chars = dat.loc[:, 'text'].apply(lambda x: list(set(x)))
    unique_chars = unique_chars.sum()
    unique_chars = sorted(np.unique(unique_chars))
    return unique_chars

In [20]:
unique_chars = get_unique_chars(dat)

In [8]:
def check_unique_chars(unique_chars, unique_tokens_filepath='unique_tokens'):
    unique_tokens = pd.read_csv(unique_tokens_filepath, header=None, index_col=[0]).iloc[:, 0]
    return set(unique_chars) - set(unique_tokens.tolist())

In [22]:
checking_outcome = check_unique_chars(unique_chars)
print(checking_outcome)

{'\x92', '@', '\x94', '´', '¡', '<', '\x97', '*', '¿', 'É', 'í', 'ñ', '\x85', '\x96', 'ú', ';', 'î', 'Í', '\x91', 'Ó', 'ó', '>', '\x93', 'á', 'à', '/', '\xa0', '&', '+', '$', 'ã', 'é', 'ç'}


In [26]:
print(dat.loc[dat.text.str.contains('@'), 'text'].iloc[0])

You're getting way too big for your boots
You're never too big for the boot
I've got the big size twelves on my feet
Your face ain't big for my boot
Kick up the yout
Man know that I kick up the yout
Dem boy dere tried twist up the truth
How dare you twist up the truth, look
You're getting way too big for your boots
You're never too big for the boot
I've got the big size twelves on my feet
Your face ain't big for my boot
Kick up the yout
Man know that I kick up the yout
Dem boy dere tried twist up the truth
How dare you twist up the truth

Wanna come round here like a badboy? Do it
Bun all the talking, go on then, do it
Running through the party, bottle of BACARDÍ
Bro's in my ear saying ""Stormz, don't do it""
Devil on my shoulder, I don't lack
Hit 'em with a crowbar, I don't scrap
Even when I'm sober, I'm so gassed
Say you ride but there's no car and no mash (what you talking 'bout?)
Clown, stand down
Never had a MAC-10 or a trey pound
You were never bad then, you ain't bad now
Never h

In [ ]:
dat = dat.text.apply(lambda x: set(list(x)))

In [21]:
if len(checking_outcome) > 0:
    append_unique_chars(checking_outcome)

Append {'\x92', '@', '\x94', '´', '¡', '<', '\x97', '*', '¿', 'É', 'í', 'ñ', '\x85', '\x96', 'ú', ';', 'î', 'Í', '\x91', 'Ó', 'ó', '>', '\x93', 'á', 'à', '/', '\xa0', '&', '+', '$', 'ã', 'é', 'ç'}


TypeError: 'set' type is unordered

In [11]:
unique_chars = pd.read_csv('unique_tokens', header=None).loc[:, 1].tolist()

In [12]:
word2ind, ind2word = utils.get_index_word_map(unique_chars)

In [13]:
vocab_size = len(unique_chars)
vocab_size

76

In [14]:
utils.save_index_word_map(word2ind, ind2word)

In [15]:
batches = dat.text.apply(len).value_counts()
lengths = batches.index.tolist()

In [16]:
for ind, length in enumerate(lengths):
    if ind % 1000 == 0: print(ind)
    cleaned = dat.loc[dat.text.apply(len) == length, 'text']
    tokenised = np.array(utils.tokenise_cleaned_data(cleaned, word2ind).tolist())
    try:
        existing = pd.read_csv('data/train_{}.csv'.format(length))
        existing.columns = list(range(length))
        increased = pd.concat([existing, pd.DataFrame(tokenised)]).drop_duplicates()
        increased.to_csv('data/train_{}.csv'.format(length), index=False)
    except:
        pd.DataFrame(tokenised).to_csv('data/train_{}.csv'.format(length), index=False)

0
1000
2000
3000
